
# Image Classification with Multiple Models on CIFAR-10 Dataset

This notebook demonstrates image classification using multiple models:
1. **Custom CNN Model**
2. **ResNet50** (Deeper ResNet variant)
3. **VGG16** (Well-known CNN architecture)
4. **DenseNet** (Model with dense connections)

We will train each model and compare their performance in terms of accuracy, loss, training time, and other key parameters.


In [ ]:

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import models

# Set device for GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:

# Define transformations: Convert to tensor and normalize
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)



### 1. Custom CNN Model

We begin by training a custom CNN model. This model has two convolutional layers, followed by two fully connected layers.


In [ ]:

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate and move the model to the device
custom_cnn = CustomCNN().to(device)



### 2. ResNet50 (Deeper ResNet Variant)

ResNet50 is a deeper architecture and has shown good performance for image classification tasks. We will fine-tune it for CIFAR-10.


In [ ]:

resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(resnet50.fc.in_features, 10)  # Modify the output layer to match CIFAR-10
resnet50 = resnet50.to(device)



### 3. VGG16

VGG16 is another deep CNN architecture that uses smaller (3x3) filters but deeper layers. We will fine-tune it for CIFAR-10.


In [ ]:

vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, 10)  # Modify the output layer to match CIFAR-10
vgg16 = vgg16.to(device)



### 4. DenseNet

DenseNet uses dense connections between layers, making it a powerful model for image classification tasks. We will fine-tune it for CIFAR-10.


In [ ]:
from torchvision import models

# For DenseNet121, use the weight enum to specify which pre-trained weights to use.
densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
densenet.classifier = nn.Linear(densenet.classifier.in_features, 10)  # Modify the output layer to match CIFAR-10
densenet = densenet.to(device)




### Model Training

Now we will train all four models and compare their performance based on accuracy, loss, and training time.


In [ ]:
# Function to train models (prints every epoch and returns full history)
def train_model(model, trainloader, testloader, num_epochs=1, lr=0.001, momentum=0.9):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    history = {
        "train_loss": [],
        "test_accuracy": []
    }

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0
        num_batches = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            num_batches += 1

        avg_train_loss = running_loss / max(1, num_batches)
        history["train_loss"].append(avg_train_loss)

        # Evaluate
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100.0 * correct / max(1, total)
        history["test_accuracy"].append(accuracy)

        # Print progress for EVERY epoch
        print(f"Epoch {epoch:>3}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Test Acc: {accuracy:.2f}%")

    return history



### Comparing Model Performance

Now, we will compare the models based on their **test accuracy** and **training loss**. We will also visualize the training progress.


In [ ]:
# Prepare models for training & comparison
models = {
    'Custom CNN': custom_cnn,
    'ResNet50': resnet50,
    'VGG16': vgg16,
    'DenseNet': densenet
}
results = {}  # stores per-epoch test accuracies for each model

In [ ]:
# Train each model (results stored in the `results` dict)
num_epochs = 2  # <- change this once; comparison/plots will reflect the full epoch count

results = {}  # model_name -> history dict

for model_name, model in models.items():
    print(f"\n==============================")
    print(f"Training {model_name} for {num_epochs} epochs")
    print(f"==============================")
    history = train_model(model, trainloader, testloader, num_epochs=num_epochs)
    results[model_name] = history


In [ ]:
# Plot test accuracy comparison across models (and print a summary table at the end)
import matplotlib.pyplot as plt
import pandas as pd

# --- Accuracy vs Epochs (explicit x-axis so it ALWAYS shows each epoch) ---
epochs = list(range(1, num_epochs + 1))

for model_name, hist in results.items():
    accs = hist["test_accuracy"]
    plt.plot(epochs, accs, label=model_name, linewidth=2, marker='o')

plt.xlabel("Epoch")
plt.ylabel("Test Accuracy (%)")
plt.title(f"Model Comparison on CIFAR-10 ({num_epochs} Epochs)")
plt.xticks(epochs)  # show EACH epoch tick
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# --- Optional: Train loss vs Epochs ---
for model_name, hist in results.items():
    losses = hist["train_loss"]
    plt.plot(epochs, losses, label=model_name, linewidth=2, marker='o')

plt.xlabel("Epoch")
plt.ylabel("Train Loss")
plt.title(f"Training Loss Comparison ({num_epochs} Epochs)")
plt.xticks(epochs)  # show EACH epoch tick
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# --- Final comparison table ---
summary_rows = []
for model_name, hist in results.items():
    accs = hist["test_accuracy"]
    losses = hist["train_loss"]
    summary_rows.append({
        "Model": model_name,
        "Epochs": len(accs),
        "Final Test Acc (%)": accs[-1] if accs else None,
        "Best Test Acc (%)": max(accs) if accs else None,
        "Final Train Loss": losses[-1] if losses else None,
        "Best Epoch (Acc)": (accs.index(max(accs)) + 1) if accs else None
    })

summary_df = pd.DataFrame(summary_rows).sort_values(by="Best Test Acc (%)", ascending=False)
print("\n===== Comparison Summary =====")
print(summary_df.to_string(index=False))
